This notebook shows how to use the new (in version 3.3) Scene interface to create custom volume renderings. To begin, we load up a dataset and use the yt.create_scene method to set up a basic Scene. We store the Scene in a variable called 'sc' and render the default ('gas', 'density') field.

In [ ]:
import yt
import numpy as np
from yt.visualization.volume_rendering.transfer_function_helper import TransferFunctionHelper
from yt.visualization.volume_rendering.api import Scene, VolumeSource

ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")
sc = yt.create_scene(ds)

Now we can look at some information about the Scene we just created using the python print keyword:

In [ ]:
print (sc)

This prints out information about the Sources, Camera, and Lens associated with this Scene. Each of these can also be printed individually. For example, to print only the information about the first (and currently, only) Source, we can do:

In [ ]:
print (sc.get_source(0))

We can see that the yt.create_source has created a VolumeSource with default values for the center, bounds, and transfer function. Now, let's see what this Scene looks like. In the notebook, we can do this by calling sc.show(). 

In [ ]:
sc.show()

That looks okay, but it's a little too zoomed-out. To fix this, let's modify the Camera associated with our Scene. This next bit of code will zoom in the camera (i.e. decrease the width of the view) by a factor of 3.

In [ ]:
sc.camera.zoom(3.0)

Now when we print the Scene, we see that the Camera width has decreased by a factor of 3:

In [ ]:
print (sc)

To see what this looks like, we re-render the image and display the scene again. Note that we don't actually have to call sc.show() here - we can just have Ipython evaluate the Scene and that will display it automatically.

In [ ]:
sc.render()
sc

That's better! The image looks a little washed-out though, so we use the sigma_clip argument to sc.show() to improve the contrast:

In [ ]:
sc.show(sigma_clip=4.0)

Next, we demonstrate how to change the mapping between the field values and the colors in the image. We use the TransferFunctionHelper to create a new transfer function using the "gist_rainbow" colormap, and then re-create the image as follows:

In [ ]:
# Set up a custom transfer function using the TransferFunctionHelper. 
# We use 10 Gaussians evenly spaced logarithmically between the min and max
# field values.
tfh = TransferFunctionHelper(ds)
tfh.set_field('density')
tfh.set_log(True)
tfh.set_bounds()
tfh.build_transfer_function()
tfh.tf.add_layers(10, colormap='gist_rainbow')

# Grab the first render source and set it to use the new transfer function
render_source = sc.get_source(0)
render_source.transfer_function = tfh.tf

sc.render()
sc.show(sigma_clip=4.0)

Now, let's try using a different lens type. We can give a sense of depth to the image by using the perspective lens. To do, we create a new Camera below. We also demonstrate how to switch the camera to a new position and orientation.

In [ ]:
cam = sc.add_camera(ds, lens_type='perspective')

# Standing at (x=0.05, y=0.5, z=0.5), we look at the area of x>0.05 (with some open angle
# specified by camera width) along the positive x direction.
cam.position = ds.arr([0.05, 0.5, 0.5], 'code_length')

normal_vector = [1., 0., 0.]
north_vector = [0., 0., 1.]
cam.switch_orientation(normal_vector=normal_vector,
                       north_vector=north_vector)

# The width determines the opening angle
cam.set_width(ds.domain_width * 0.5)

print (sc.camera)

The resulting image looks like:

In [ ]:
sc.render()
sc.show(sigma_clip=4.0)

Finally, the next cell restores the lens and the transfer function to the defaults, moves the camera, and adds an opaque source  that shows the axes of the simulation coordinate system.

In [ ]:
# set the lens type back to plane-parallel
sc.camera.set_lens('plane-parallel')

# move the camera to the left edge of the domain
sc.camera.set_position(ds.domain_left_edge)
sc.camera.switch_orientation()

# add an opaque source to the scene
sc.annotate_axes()

sc.render()
sc.show(sigma_clip=4.0)